# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
!pip install langchain


In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

¡Claro! En el contexto de modelos de lenguaje como los que utiliza LangChain, la temperatura es un parámetro que controla la aleatoriedad o creatividad en las respuestas generadas por el modelo.

¿Cómo funciona la temperatura?
Valores bajos de temperatura (ej. 0.0 a 0.3):

El modelo se vuelve más determinista y preciso.
Es ideal para tareas en las que necesitas respuestas claras y repetibles, como cálculos matemáticos, responder preguntas factuales o generar texto técnico.
Valores intermedios (ej. 0.5 a 0.7):

El modelo introduce algo de variabilidad.
Es útil para tareas que requieren un balance entre creatividad y precisión, como escribir respuestas en lenguaje natural.
Valores altos (ej. 0.8 a 1.0 o más):

El modelo genera respuestas más creativas y diversas, pero puede volverse menos preciso.
Es ideal para tareas como generación de ideas, escritura creativa o exploración de múltiples posibilidades.

In [9]:
from dotenv import load_dotenv
import os

load_dotenv()
llm = ChatOpenAI(temperature=0.3, openai_api_key=os.getenv("OPENAI_API_KEY"))


In [10]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.3)


In [11]:
prompt = ChatPromptTemplate.from_template( "Describe the key features and benefits of the product: {product_name}."
 
)

In [12]:

chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/ln/h2lpkllx5fl21m8307tnfsdc0000gn/T/ipykernel_68894/546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [13]:
product = 'wireless headphones'
chain.run(product)

/var/folders/ln/h2lpkllx5fl21m8307tnfsdc0000gn/T/ipykernel_68894/965846579.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'Wireless headphones are a popular audio accessory that allows users to listen to music, podcasts, and other audio content without being tethered to their device by a cord. Key features of wireless headphones include:\n\n1. Bluetooth connectivity: Wireless headphones use Bluetooth technology to connect to devices such as smartphones, tablets, and computers, allowing for a seamless and wireless listening experience.\n\n2. Comfort and convenience: Wireless headphones are typically lightweight and designed for comfort, making them ideal for extended use. They also eliminate the hassle of dealing with tangled cords.\n\n3. Noise cancellation: Many wireless headphones come equipped with noise-cancelling technology, which helps to block out background noise and provide a more immersive listening experience.\n\n4. Long battery life: Wireless headphones often have a long battery life, allowing users to enjoy hours of uninterrupted listening before needing to recharge.\n\n5. Portability: Wireles

## SimpleSequentialChain

In [22]:
from langchain.chains import SimpleSequentialChain

In [28]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Describe the main features of the product: {product_name}."
)


# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="product_description")


In [29]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the description, suggest three potential use cases for this product."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.run(product)

**Repeat the above twice for different products**

## SequentialChain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following review into Italian: {review}."
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translated_review" #Give a name to your output
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review: {translated_review}."
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="review_summary" #give a name to this output
                    )


In [ ]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
   "Translate the following summary into Spanish: {review_summary}."
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key=N"translated_summary"
                      )


In [ ]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Based on the translated review and summary, write a follow-up message for the reviewer. "
    "Translated review: {translated_review}. "
    "Summary: {review_summary}."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [ ]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["translated_review", "review_summary", "translated_summary", "followup_message"],
    verbose=True
)

In [ ]:
review = df.Review[5]
overall_chain(review)

**Repeat the above twice for different products or reviews**

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
What is the difference between classical mechanics and quantum mechanics?
"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question: How do you calculate the area under a curve using integration?
"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
What were the main causes of World War I?
"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
What is the difference between a stack and a queue in data structures?
"""

In [ ]:
prompt_infos = [
    {
        "name": "Physics",
        "description": "A prompt designed to answer questions about physics in a concise and accurate way.",
        "prompt_template": "You are a physics professor. Answer the following question: {input}"
    },
    {
        "name": "Math",
        "description": "A prompt designed to answer complex mathematical questions.",
        "prompt_template": "You are a mathematician. Solve the following problem: {input}"
    },
    {
        "name": "History",
        "description": "A prompt designed to answer questions about history and historical events.",
        "prompt_template": "You are a historian. Provide insights about the following: {input}"
    },
    {
        "name": "Computer Science",
        "description": "A prompt designed to answer questions about computer science and coding.",
        "prompt_template": "You are a computer scientist. Explain the following topic: {input}"
    }
]


In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [ ]:
chain.run("What is black body radiation?")  # Física
chain.run("What is 2 + 2?")  # Matemáticas
chain.run("Why does every cell in our body contain DNA?")  # Biología (podría ir a default_chain si no está definido)


In [ ]:
# Prueba con varias preguntas
result1 = chain.run("What is black body radiation?")
print("Physics Chain Output:", result1)

result2 = chain.run("What is 2 + 2?")
print("Math Chain Output:", result2)

result3 = chain.run("Why does every cell in our body contain DNA?")
print("Biology/Default Chain Output:", result3)
